In [18]:
'''
'Title' - название позиции ✔️
'SpecCat' - спциализация 
'ExpPeriod' - опыт ✔️
'Salary' - ожидаемая зп ✔️
'Age' - возраст ✔️
'Gender' - пол ✔️
'City' - город ✔️
'WorkType' - тип работы ✔️
'WorkSchedule' - график занятости ✔️
'N_places' - количество мест работы ✔️
'LastPlace' - последнее место работы 1 ✔️-
'LastPlace_2' - последнее место работы 2 ✔️-
'LastPos' - позиция на последнем месте работы 1 ✔️-
'LastPos_2' - позиция на последнем месте работы 2 ✔️-
'N_langs' - количество языков владения ✔️
'Langs' - список языков ✔️
'Citizenship' - гражданство  ✔️
'Time2Work' - время до работы ✔️
'WorkRights' - обязанности на работе ✔️
'Ready_2_move' - флаг ‘готов к переезду’ ✔️
'Ready_4_business_trip' - флаг ‘готов к командировкам’
'Description' - личное описание кандидата ✔️
'Tags' - теги (основные навыки и интересы) ✔️
'Graduation' - образование
'EduLevel' - уровень образования 
'LastUni' - последний университет 1 ✔️
'LastUni_2' - последний университет 2 ✔️
'N_Unis' - количество мест, где человек обучался ✔️
'URL' ✔️
'''

"\n'Title' - название позиции ✔️\n'SpecCat' - спциализация \n'ExpPeriod' - опыт ✔️\n'Salary' - ожидаемая зп ✔️\n'Age' - возраст ✔️\n'Gender' - пол ✔️\n'City' - город ✔️\n'WorkType' - тип работы ✔️\n'WorkSchedule' - график занятости ✔️\n'N_places' - количество мест работы ✔️\n'LastPlace' - последнее место работы 1 ✔️-\n'LastPlace_2' - последнее место работы 2 ✔️-\n'LastPos' - позиция на последнем месте работы 1 ✔️-\n'LastPos_2' - позиция на последнем месте работы 2 ✔️-\n'N_langs' - количество языков владения ✔️\n'Langs' - список языков ✔️\n'Citizenship' - гражданство  ✔️\n'Time2Work' - время до работы ✔️\n'WorkRights' - обязанности на работе ✔️\n'Ready_2_move' - флаг ‘готов к переезду’ ✔️\n'Ready_4_business_trip' - флаг ‘готов к командировкам’\n'Description' - личное описание кандидата ✔️\n'Tags' - теги (основные навыки и интересы) ✔️\n'Graduation' - образование\n'EduLevel' - уровень образования \n'LastUni' - последний университет 1 ✔️\n'LastUni_2' - последний университет 2 ✔️\n'N_Unis'

# Import Libraries #

In [19]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [20]:
import time
import pandas as pd
import numpy as np
import re
from random import uniform
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
from csv import DictWriter
from fake_useragent import UserAgent
from urllib.request import urlopen, Request
from urllib.error import HTTPError, URLError
from datetime import date
from dateutil.relativedelta import *
from datetime import datetime

# Fetching Data #

In [21]:
FILE_NAME = 'resumes_raw.csv'

PROFESSION_NAMES = [
  'finansovyy_analitik',
  'sistemnyy_analitik',
  'analitik_prodazh',
  'biznes_analitik',
  'analitik-dannyh',
  'data-analyst',
  'analitik-bi'
]

FEATURE_NAMES = [
  'Title', 'SpecCat', 'ExpPeriod', 'Salary', 'Age', 'Gender', 'City', 'WorkType', 'WorkSchedule', 
  'WorkExp', 'N_places', 'N_places_last_year', 'LastPlace', 'LastPlace_2', 'LastPos', 'LastPos_2', 'LastExp', 
  'LastExp_2', 'LastExpDesc', 'LastExpDesc_2', 'N_langs', 'Langs', 'ResumeLang', 'Citizenship', 'Time2Work', 
  'WorkRights', 'Description', 'Tags', 'Graduation', 'EduLevel', 'LastUni', 'LastUni_2', 'N_Unis', 'URL'
]

BASE_URL = 'https://hh.ru'

In [22]:
try:
  df = pd.read_csv(FILE_NAME, low_memory=False)
  df['URL']
except:
  df = pd.DataFrame(data=[], columns=FEATURE_NAMES)
  df.to_csv(FILE_NAME, index=False)

time.sleep(3)


In [23]:
ua = UserAgent()

In [24]:
def get_max_page(url):
  user_agent = ua.random
    
  try:
    req = Request(
      url, 
        data=None, 
        headers={'User-Agent': user_agent}
    )
    html = urlopen(req).read()
  except HTTPError as e:
    print('HTTPError: {}'.format(e.code))
  except URLError as e:
    print('URLError: {}'.format(e.reason))
  else:
    soup = BeautifulSoup(html, features="html.parser")
    try:
      a_href = soup.find_all('a', { 'class' : 'bloko-button'})[-2].get('href')
      max_page = re.search('(\d+)', a_href).group(1)
    except:
      max_page = '0'
  return int(max_page)

In [25]:
def get_exp(array = []):
  if array:
    period = 0
    for element in array:
      if ('лет' in element) or ('год' in element) or ('year' in element):
        period += int(element.split()[0]) * 12
      if ('мес') in element or ('mon' in element):
        period += int(element.split()[0])
      return period
  return None

In [26]:
def get_age(string = ''):
  if string:
    try:
      return int(string.split()[0])
    except:
      return None
  return None    

In [27]:
def get_resume_lang(string = ''):
  if string:
    if bool(re.search('[а-яА-Я]', string)):
      return 'ru'
    if bool(re.search('[a-zA-Z]', string)):
      return 'en'
  return None

In [28]:
def get_places_last_year(arr):
  months_dict = {
    'january': 1,
    'february': 2,
    'march': 3,
    'april':4,
    'may':5,
    'june':6,
    'july':7,
    'august':8,
    'september':9,
    'october':10,
    'november':11,
    'december':12,
    
    'январь': 1,
    'февраль': 2,
    'март': 3,
    'апрель':4,
    'май':5,
    'июнь':6,
    'июль':7,
    'август':8,
    'сентябрь':9,
    'октябрь':10,
    'ноябрь':11,
    'декабрь':12
    }

  count = 0
  current_month = datetime.now().month
  current_year = datetime.now().year

  for pair in arr:
    if pair == (' ', ' '):
      count += 1
    try:
      if (current_year * 12 + current_month) - (int(pair[1]) * 12 + months_dict[pair[0].lower()]) <= 12:
        count += 1
    except:
      count
  return count

In [29]:
def get_resume(url):
  user_agent = ua.random
    
  try:
    req = Request(
      url, 
      data=None, 
      headers={'User-Agent': user_agent}
    )
    html = urlopen(req).read()
  except HTTPError as e:
    print('HTTPError: {}'.format(e.code))
  except URLError as e:
    print('URLError: {}'.format(e.reason))
  else:
    soup = BeautifulSoup(html, features="html.parser")
  
    # Title
    try:
      title = soup.find(attrs={'class': 'resume-block__title-text', 'data-qa': 'resume-block-title-position'}).text
    except:
      title = None
        
    # Specialization
    try:
      spec_list = soup.findAll(attrs={'class':'resume-block__specialization'})
      specializations = [s.text for s in spec_list]
    except:
      specializations = []
    
    # Experience
    try:
      whole_exp_spans = soup.find(attrs={'class':'resume-block__title-text resume-block__title-text_sub'})
      whole_exp_period = get_exp([s.text.replace('\xa0', ' ') for s in whole_exp_spans.findChildren('span')])
      whole_exp_period = int(whole_exp_period)
    except:
      whole_exp_period = None
    
    # Salary
    try:
      salary = soup.find(attrs={'class':'resume-block__salary resume-block__title-text_salary'}).text
      salary = salary.replace('\xa0', '').replace('\u2009', '')
    except:
      salary = None

    
    ps = soup.findAll('p')
    p = [p.text for p in ps]
        
    # Gender
    try:
      gender_string = soup.find(attrs={'data-qa': 'resume-personal-gender'}).text.lower()
      resume_lang = get_resume_lang(gender_string)
      gender = gender_string
    except:
      gender = None
      resume_lang = None
    
    # Age
    try:
      age = soup.find(attrs={'data-qa': 'resume-personal-age'}).text.replace('\xa0', ' ')
      age = get_age(age)
    except:
      age = None
    
    # Address
    try:
      address = soup.find(attrs={'data-qa': 'resume-personal-address'}).text
    except:
      address = None
    
    # Work type and schedule
    try:
      work_type = ''
      try:
        work_type += list(filter(lambda x: ('занятость') in x.lower(), p))[0]
      except:
        work_type += list(filter(lambda x: ('employment') in x.lower(), p))[0]
    except:
      work_type = None

    try:
      work_schedule = ''
      try:
        work_schedule += list(filter(lambda x: ('график') in x.lower(), p))[0]
      except:
        work_schedule += list(filter(lambda x: ('schedule') in x.lower(), p))[0]
    except:
      work_schedule = None
    
    # experience block
    try:
      work_exp_periods = []
      exp_block = soup.select_one('div[data-qa="resume-block-experience"]')
      try:
        work_exp_period_divs = exp_block.select('div[class="bloko-column bloko-column_xs-4 bloko-column_s-2 bloko-column_m-2 bloko-column_l-2"]')
        exp_dates_text = [div.find_all(text=True) for div in work_exp_period_divs]
        exp_dates = [(arr[7].lower(), arr[10]) for arr in exp_dates_text]
        n_places_last_year = get_places_last_year(exp_dates)
                
        work_exp_period_spans = [divs.find('div').findAll('span') for divs in work_exp_period_divs]
        for period in work_exp_period_spans:
          work_exp_periods.append(get_exp([p.text.replace('\xa0', ' ') for p in period]))
      except:
        n_places_last_year = 0
        work_exp_periods = None

      try:
        work_place_pos_divs = exp_block.select('div[class="bloko-column bloko-column_xs-4 bloko-column_s-6 bloko-column_m-7 bloko-column_l-10"]')
        work_places = [div.find('div').find(attrs={'class':'bloko-text bloko-text_strong'}).text for div in work_place_pos_divs]
        n_places = len(work_places)
      except:
        work_places = []
        n_places = None
            
      try:
        work_place_pos_divs = exp_block.select('div[class="bloko-column bloko-column_xs-4 bloko-column_s-6 bloko-column_m-7 bloko-column_l-10"]')
        work_positions = [div.select_one('div[data-qa="resume-block-experience-position"]').text for div in work_place_pos_divs]
      except:
        work_positions = []

      try:
        work_place_pos_divs = exp_block.select('div[class="bloko-column bloko-column_xs-4 bloko-column_s-6 bloko-column_m-7 bloko-column_l-10"]')
        exp_description = [div.select_one('div[data-qa="resume-block-experience-description"]').text for div in work_place_pos_divs]
      except:
        exp_description = []
            

    except:
      work_exp_periods = []
      work_places = []
      work_positions = []
      n_places = None
    
    # Places, Experience and Positions
    try:
      last_place = work_places[0]
      last_position = work_positions[0]
      last_exp = work_exp_periods[0]
      last_exp_desc = exp_description[0]
    except:
      last_place = None
      last_position = None
      last_exp = None
      last_exp_desc = None
        
    try:
      last_place_2 = work_places[1]
      last_position_2 = work_positions[1]
      last_exp_2 = work_exp_periods[1]
      last_exp_desc_2 = exp_description[1]
    except:
      last_place_2 = None
      last_position_2 = None
      last_exp_2 = None
      last_exp_desc_2 = None
    
    # Description
    try:
      description = soup.select_one('div[data-qa="resume-block-skills-content"]').text
    except:
      description = None
    
    # Languages
    try:
      languages = soup.select('p[data-qa="resume-block-language-item"]')
      languages = [p.text for p in languages]
      n_lang = len(languages)
    except:
      languages = None
      n_lang = None
    
    # Legal bio
    try:
      citizenship = ''
      try:
        citizenship += list(filter(lambda x: ('гражданство') in x.lower(), p))[0]
      except:
        citizenship += list(filter(lambda x: ('citizenship') in x.lower(), p))[0]
    except:
      citizenship = None
        
    try:
      work_rights = ''
      try:
        work_rights += list(filter(lambda x: ('разрешение') in x.lower(), p))[0]
      except:
        work_rights += list(filter(lambda x: ('permission') in x.lower(), p))[0]
    except:
      work_rights = None
        
    try:
      time_to_work = ''
      try:
        time_to_work += list(filter(lambda x: ('желательное') in x.lower(), p))[0]
      except:
        time_to_work += list(filter(lambda x: ('desired') in x.lower(), p))[0]
    except:
      time_to_work = None
            
    
    # Tags
    try: 
      tags_list = soup.find('div', {'class' : 'bloko-tag-list'})
      tags = tags_list.find_all('div')
      tags_texts = [tag.find('span').text for tag in tags]
    except:
      tags_texts = []

    # Graduation
    try:
      grad_block = soup.select_one('div[data-qa="resume-block-additional-education"]')
      try:
        grad_div = grad_block.find('div', {'class' : "resume-block-item-gap"}).find('div').find('div')
        grad_text_div = grad_div.find('div', {'class' : 'bloko-column bloko-column_xs-4 bloko-column_s-6 bloko-column_m-7 bloko-column_l-10'})
        graduation = grad_text_div.find('div').find('div').text
      except:
        graduation = None
    except:
      graduation = None
    
    # Education
    try:
      edu_block = soup.select_one('div[data-qa="resume-block-education"]')
      edu_rows = edu_block.find('div', {'class' : 'resume-block-item-gap'}).find('div')
      edu_list = [item.find('div') for item in edu_rows.findChildren('div' , recursive=False)]
      edu_items = [item.findChildren('div' , recursive=False)[-1].find('div').find('div').text for item in edu_list]
    
      edu_level = edu_block.find('span').text
    
      n_unis = len(edu_items)
    except:
      edu_level = None
      n_unis = None
    
    try:
      last_uni = edu_items[0]
    except:
      last_uni = None
    
    try:
      last_uni_2 = edu_items[1]
    except:
      last_uni_2 = None
    
    # URL
    URL = url
    
    return {
      'Title': title,
      'SpecCat': specializations,
      'ExpPeriod': whole_exp_period,
      'Salary' : salary,
      'Age' : age,
      'Gender' : gender,
      'City' : address,
      'WorkType' : work_type,
      'WorkSchedule' : work_schedule,
      'WorkExp' : work_exp_periods,
      'N_places' : n_places,
      'N_places_last_year' : n_places_last_year,
      'LastPlace' : last_place, 
      'LastPlace_2' : last_place_2,
      'LastPos' : last_position,
      'LastPos_2' : last_position_2,
      'LastExp' : last_exp,
      'LastExp_2' : last_exp_2,
      'LastExpDesc' : last_exp_desc,
      'LastExpDesc_2' : last_exp_desc_2,
      'N_langs' : n_lang,
      'Langs' : languages,
      'ResumeLang' : resume_lang,
      'Citizenship' : citizenship,
      'Time2Work' : time_to_work, 
      'WorkRights' : work_rights, 
      'Description' : description, 
      'Tags' : tags_texts, 
      'Graduation' : graduation, 
      'EduLevel' : edu_level, 
      'LastUni' : last_uni, 
      'LastUni_2' : last_uni_2, 
      'N_Unis' : n_unis, 
      'URL' : URL
    }

In [30]:
# get_resume('https://hh.ru/resume/02c4130b00050d82080039ed1f7a3634376a75?query=Bi+%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA%D0%B0&hhtmFrom=resume_search_result')

In [31]:
def write_row_to_csv(resume_dict):
  with open(FILE_NAME, 'a') as f_object:
    dictwriter_object = DictWriter(f_object, fieldnames=FEATURE_NAMES)
    dictwriter_object.writerow(resume_dict)
    f_object.close()

In [32]:
def get_resumes_at_page(url, page, initial_urls):
  page_url = url + '?page={}'.format(page)
  html = urlopen(page_url).read()
  soup = BeautifulSoup(html, features="html.parser")

  for link in soup.find_all('a', href=True):
    if 'query' in link['href']:
      url = BASE_URL + link['href']
      if url not in initial_urls:
        resume = get_resume(url)
        write_row_to_csv(resume)
    
  return True

In [33]:
def download_profession(name):
  df = pd.read_csv(FILE_NAME, low_memory=False)
  url = '{}/resumes/{}'.format(BASE_URL, name)

  for page in tqdm(range(0, get_max_page(url) + 1), desc=f'Profession: {name}'):
    get_resumes_at_page(url, page, df['URL'].values)
    time.sleep(uniform(0.0005, 0.002))

In [35]:
for name in tqdm(PROFESSION_NAMES, desc=f'No of professions downloaded'):
  download_profession(name)

No of professions downloaded:   0%|          | 0/7 [00:00<?, ?it/s]

Profession: finansovyy_analitik:   0%|          | 0/250 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
df = df.drop_duplicates('URL', keep='last')
df.to_csv(FILE_NAME, index=False)

# Cleaning Data #

In [ ]:
from ast import literal_eval
pd.set_option('display.max_columns', None)

In [ ]:
all_df = pd.read_csv(FILE_NAME, low_memory=False)
all_df = all_df.drop_duplicates('Description', keep='last')

In [ ]:
ru_df = all_df[(all_df['ResumeLang'] == 'ru') & (all_df['URL'] != None)]
ru_df = ru_df.copy(deep=True)
ru_df = ru_df.reset_index(drop=True)

In [ ]:
ru_df

,Title,SpecCat,ExpPeriod,Salary,Age,Gender,City,WorkType,WorkSchedule,WorkExp,N_places,N_places_last_year,LastPlace,LastPlace_2,LastPos,LastPos_2,LastExp,LastExp_2,LastExpDesc,LastExpDesc_2,N_langs,Langs,ResumeLang,Citizenship,Time2Work,WorkRights,Description,Tags,Graduation,EduLevel,LastUni,LastUni_2,N_Unis,URL
0,Финансовый аналитик,"['Финансовый менеджер', 'Финансовый аналитик, ...",NaN,NaN,35.0,мужчина,Москва,Занятость: полная занятость,График работы: полный день,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,['Английский — C2 — В совершенстве'],ru,Гражданство: Россия,Желательное время в пути до работы: Не имеет з...,NaN,Уверенный пользователь ПК\r\nКоммуникабельност...,[],NaN,Высшее образование,ФА -- Финансовая академия при Правительстве РФ,NaN,1.0,https://hh.ru/resume/c4912d470004c8cba90039ed1...
1,Финансовый аналитик,"['Финансовый аналитик, инвестиционный аналитик...",NaN,NaN,21.0,женщина,Москва,"Занятость: частичная занятость, полная занятость","График работы: удаленная работа, гибкий график...",[29],1.0,1.0,Air Liquide,NaN,Бухгалтер по затратам,NaN,29.0,NaN,- корректное и своевременное отражение бухгалт...,NaN,2.0,"['Русский — Родной', 'Английский — A2 — Элемен...",ru,Гражданство: Россия,Желательное время в пути до работы: Не имеет з...,Разрешение на работу: Россия,Интересует работа с отчетами и цифрами. В бухг...,"['Аналитическое мышление', 'Точность и внимате...",NaN,Неоконченное высшее образование,Российский экономический университет им. Г.В. ...,ВФ РЭУ им.Плеханова,2.0,https://hh.ru/resume/3a2836470007ebfcf00039ed1...
2,"Менеджер проектов, финансовый аналитик",['Руководитель проектов'],186.0,150000руб.,44.0,женщина,Москва,Занятость: полная занятость,График работы: полный день,"[40, 49, 19, 85]",4.0,1.0,Good Foods,"Emerald resorts, Bulgaria",Финансовый аналитик,Финансовый директор,40.0,49.0,"Создание и реализация проекта с нуля, полный ...",Полный финансовый контроль пятизвездочного оте...,3.0,"['Русский — Родной', 'Английский — C1 — Продви...",ru,Гражданство: Россия,Желательное время в пути до работы: Не имеет з...,Разрешение на работу: Россия,"Исполнительная, ответственная, целеустремленна...",['Исполнительность'],NaN,Высшее образование,Московская государственная финансовая академия,Красноярский государственный торгово-экономиче...,2.0,https://hh.ru/resume/bb88768d000756bc0e0039ed1...
3,Ведущий экономист финансового отдела,['Экономист'],269.0,130000руб.,44.0,мужчина,Москва,Занятость: полная занятость,График работы: полный день,"[109, 7, 70, 48, 37]",5.0,1.0,"ООО ""Итепло""","Вэб Медиа Сервисез, ЗАО",Финансовый аналитик,Руководитель отдела логистики и закупок,109.0,7.0,"Формирование бюджета, включая 8 обособленных п...","Организация, контроль и планирование работы по...",2.0,"['Русский — Родной', 'Английский — C1 — Продви...",ru,Гражданство: Россия,Желательное время в пути до работы: Не имеет з...,Разрешение на работу: Россия,Знание компьютера на уровне эксперт;\r\nMicros...,"['Ответственность', 'Высокий уровень грамотнос...",Практический курс 1С Бухгалтерия 8.0,Высшее образование,Финансовый Университет при Правительтве РФ,Томский Университет Систем Управления и Радиоэ...,2.0,https://hh.ru/resume/329c60e500011c9d6b0039ed1...
4,Методолог по бюджетированию и управленческому ...,['Финансовый менеджер'],327.0,NaN,51.0,женщина,Москва,"Занятость: частичная занятость, проектная рабо...","График работы: гибкий график, полный день, уда...","[43, 4, 9, 86, 23, 57, 108]",7.0,1.0,Индивидуальное предпринимательство / частная п...,ООО Агрохолдинг,Финансовый аналитик,Главный специалист по бюджетированию и управле...,43.0,4.0,В последнее время в связи с пандемией занялась...,"Постановка управленческого учета , внедрение п...",2.0,"['Русский — Родной', 'Английский — A1 — Началь...",ru,Гражданство: Россия,Желательное время в пути до работы: Не более часа,Разрешение на работу: Россия,"Имею опыт работы на производстве, оптовой и ро...","['Управление затратами', 'Бюджетирование', 'Фи...",Институт русского предпринимательства,Высшее об

In [ ]:
np.set_printoptions(threshold=np.inf)
ru_df.LastPlace.unique()

array([nan, 'Air Liquide', 'Good Foods ', 'ООО "Итепло"',
       'Индивидуальное предпринимательство / частная практика / фриланс',
       'ООО "Транс Нафта Газдобыча"', 'KPMG', 'РЖД-Инвест', 'Яндекс',
       'ГК Мэдокс', 'Sucden', 'EY Russia', 'ООО "Метзлер"',
       'ООО "СилверК 21"', 'Частная практика', 'Toyota Motor',
       'PricewaterhouseCoopers (Россия)',
       'ИНТЕРЭКСПЕРТИЗА, аудиторская компания', 'KPMG ',
       'Egger Drevprodukt Shuya', 'Kept (ранее - KPMG)',
       'ОЭБ и ПК по г. Костроме', 'Сбер', 'ТОО "Приоритет Консалтинг"',
       'Tsukiji.group', 'ООО СпецСтрой Транспорт', 'Сбербанк',
       'МегаФон, ПАО', 'Mail.Ru Group', 'Консультационная работа',
       'PwC (PricewaterhouseCoopers)', 'ООО УК «Деловой альянс»',
       'ООО Шато де Талю', 'Аудитор на подряд', 'АО Тандер',
       'АО «Корпорация Попов Радио»', 'Sberbank CIB',
       'Европейский трастовый банк',
       'Фонд капитального ремонта города Москвы', 'ООО "Домашние Деньги"',
       'Энергохолдинг', 

In [ ]:
def trim_and_divide(text, trim, sep, lower=False):
  if isinstance(text, str):
    try:
      text = str(text)
      if lower:
        text = text.lower()
      for i in trim:
        text = text.replace(i, '')
        res = text.split(sep)
      return res
    except:
      return np.nan
  return np.nan

In [ ]:
def map_dict_to_array(dict, array):
  if array:
    try:
      res = [dict[element] for element in array]
      res.sort()
      return res
    except:
      return np.nan
  return np.nan

In [ ]:
def get_work_schedule_cat(string):
  res = {}
  arr = trim_and_divide(string, ['график работы: ', 'work schedule: '], ', ', True)
  arr = map_dict_to_array(
    array=arr,
    dict={
      'вахтовый метод': None,
      'rotation based work': None,
      'удаленная работа': 'remote',
      'remote working': 'remote',
      'гибкий график': None,
      'flexible schedule': None, 
      'полный день': 'fullday',
      'full day': 'fullday', 
      'сменный график': None,
      'shift schedule': None
    }
  )
  if type(arr) is list:
    for element in arr:
      if element == 'remote':
        res['WorkSchedule_Remote'] = 1
      elif element == 'fullday':
        res['WorkSchedule_FullDay'] = 1
  return res
    

In [ ]:
def get_any_time_to_work(string):
  arr = trim_and_divide(string, ['желательное время в пути до работы: ', 'desired travel time to work: '], ', ', True)
  res = map_dict_to_array(
    array=arr,
    dict={
      'не имеет значения': 1,
      "doesn't matter": 1,
      'не более часа': 0,
      'up to one hour': 0,
      'не более полутора часов': 0,
      'up to 90 minutes': 0,
    }
  )
  if 1 in res:
    return 1
  else:
    return 0

In [ ]:
def get_fulltime_work(string):
  arr = trim_and_divide(string, ['занятость: ', 'employment: '], ', ', True)
  res = map_dict_to_array(
    array=arr,
    dict={
      'волонтерство': 0,
      'volunteering': 0,
      'стажировка': 0,
      'work placement': 0,
      'частичная занятость': 0,
      'part time': 0,
      'проектная работа': 0,
      'project work': 0,
      'полная занятость': 1,
      'full time': 1
    }
  )
  if 1 in res:
    return 1
  else:
    return 0

    

In [ ]:
def get_salary(string):
  if string:
    string = str(string).replace('.', '').upper()
    try:
      mess = re.match('([0-9]+)([a-zA-Z]+|[а-яА-Я]+)', string)
      value = int(mess.group(1))
      currency = mess.group(2)
      if (currency == 'РУБ') or (currency == 'RUB'):
        return value
      elif currency == 'USD':
        return value * 65
      elif currency == 'EUR':
        return value * 70
    except: 
      return np.nan
  return np.nan

In [ ]:
def get_lang_level(string):
  dict_ru = { 
    "A1":0, 
    "A2":0,
    "B1":0, 
    "B2":0,
    "C1":0,
    "C2":0,
    "Родной":1,
    "Native":1
  }

  dict_en = { 
    "A1":0, 
    "A2":0,
    "B1":0, 
    "B2":0,
    "C1":1,
    "C2":1,
    "Родной":1,
    "Native":1
  }
    
  if isinstance(string, str):
    res = {}
    try:
      arr = literal_eval(string)
      for element in arr:
        item = element.split(' — ')
        if (item[0] == 'Русский') or (item[0] == 'Russian'):
          res['Language_RU_Native'] = dict_ru[item[1]]
        if (item[0] == 'Английский') or (item[0] == 'English'):
          res['Language_EN_Advanced'] = dict_en[item[1]]
      if res:
        return res
      else:
        return np.nan
    except:
      return np.nan
  return np.nan

In [ ]:
def get_gender(string):
  if string:
    if ('мужчина' in str(string)) or ('male' in str(string)):
      return 1
    elif ('женщина' in str(string)) or ('female' in str(string)):
      return 0
    else:
      return np.nan
  return np.nan

In [ ]:
def get_citizenship(string):
  res = {}
  arr = trim_and_divide(string, ['Гражданство: ', 'Citizenship: '], ', ')
  if type(arr) is list:
    for element in arr:
      if element == 'Россия':
        res['Citizenship_Russia'] = 1
      elif element == 'Беларусь':
        res['Citizenship_Belarus'] = 1
      elif element == 'Казахстан':
        res['Citizenship_Kazakhstan'] = 1
  return res

In [ ]:
def get_work_rights(string):
  res = {}
  arr = trim_and_divide(string, ['Разрешение на работу: ', 'Permission to work: '], ', ')
  if type(arr) is list:
    for element in arr:
      if element == 'Россия':
        res['WorkRights_Russia'] = 1
      elif element == 'Беларусь':
        res['WorkRights_Belarus'] = 1
      elif element == 'Казахстан':
        res['WorkRights_Kazakhstan'] = 1
  return res

# {'WorkRights_Russia':1, ...}

In [ ]:
def get_education(education_level):
  res = {}
  if type(education_level) is str:
    if 'Высшее образование'.lower() in education_level.lower():
      res['hasHigherSchoolFinished'] = 1
    
    if 'Бакалавр'.lower() in education_level.lower():
      res['hasHigherSchoolFinished'] = 1
      res['hasBachelorDeg'] = 1
      res['hasMastersDeg'] = 0
      res['hasDoctorDeg'] = 0

    if 'Магистр'.lower() in education_level.lower():
      res['hasHigherSchoolFinished'] = 1
      res['hasBachelorDeg'] = 1
      res['hasMastersDeg'] = 1
      res['hasDoctorDeg'] = 0
    
    if ('Кандидат наук'.lower() in education_level.lower()) or ('Доктор наук'.lower() in education_level.lower()):
      res['hasHigherSchoolFinished'] = 1
      res['hasBachelorDeg'] = 1
      res['hasMastersDeg'] = 1
      res['hasDoctorDeg'] = 1
    
  return res

In [ ]:
def check_top_15_unis(uni):
  universities = {
    "Московский Государственный Университет", "МГУ", #1
    "ИТМО", #2
    "Московский физико-технический институт", "МФТИ", "Физтех", #3
    "Высшая школа экономики", "ВШЭ", "Санкт-Петербургский государственный университет", #4
    "Московский государственный технический университет им. Н.Э.Баумана", "Баумана", "Бауманка", "Бауманский", "МГТУ", "МВТУ", #5
    "МЭИ", "Московский энергетический институт", #6
    "МИСиС", #7
    "Московский инженерно-физический институт", "МИФИ" #8
    "Санкт-Петербургский политехнический университет Петра Великого", "СПбПУ" #9
    "Санкт-Петербургский государственный университет", "СПбГУ", #10
    "Национальный исследовательский томский политехнический университет", "ТПУ", #11
    "Новосибирский государственный университет", "НГУ", #12
    "Московский институт электроники и математики им. А.Н. Тихонова", "МИЭМ", "Тихонова"
    # екатеринбургский
  }
    
  flag = 0
  for i in universities:
    if type(uni) is str:
      if i.lower() in uni.lower():
        flag = 1
  return flag

In [ ]:
def check_top_20_places(place):
  places = {
    "ВТБ", "Вымпелком", "Мегафон", "МТС", "Авито", "ВК",
    "Росатом", "Ростелеком", "Сбер", "Тинькофф", "Яндекс", "VTB", "Альфа-Банк",
    "Avito", "VK", "X5 retail", "Х5 retail", "Касперск", "Ozon", "Yandex", "Oзон", 
    "Raiffeisenbank", "Райффайзенбанк", "Газпром", 
    "ВТБ", "VTB" "Apple", "Google", "Mail.Ru", "МегаФон", "Intel", "Citibank"
  }
    
  flag = 0
  for i in places:
    if type(place) is str:
      if i.lower() in place.lower():
        flag = 1
  return flag

In [ ]:
def get_exp(string):
  try:
    return float(string)
  except:
    return np.nan

In [ ]:
def get_arr(string):
  try:
    arr = literal_eval(string)
    return ', '.join(arr)
  except:
    return np.nan

In [ ]:
ru_df = ru_df.rename(columns={"ExpPeriod": "ExpPeriod_Months", "LastExp": "LastExp_Months", "LastExp_2": "LastExp_2_Months"})

In [ ]:
ru_df['SpecCat'] = ru_df['SpecCat'].apply(get_arr)
ru_df['Gender'] = ru_df['Gender'].apply(get_gender)
ru_df['Salary'] = ru_df['Salary'].apply(get_salary)
ru_df['EduLevel'] = ru_df['EduLevel'].apply(get_education)
ru_df['FullTimeWork'] = ru_df['WorkType'].apply(get_fulltime_work)
ru_df['WorkSchedule'] = ru_df['WorkSchedule'].apply(get_work_schedule_cat)
ru_df['Time2Work_Any'] = ru_df['Time2Work'].apply(get_any_time_to_work)
ru_df['WorkRights'] = ru_df['WorkRights'].apply(get_work_rights)
ru_df['Citizenship'] = ru_df['Citizenship'].apply(get_citizenship)
ru_df['Langs'] = ru_df['Langs'].apply(get_lang_level)
ru_df['Tags'] = ru_df['Tags'].apply(get_arr)
ru_df['Top12LastUni'] = ru_df['LastUni'].apply(check_top_15_unis)
ru_df['Top12LastUni_2'] = ru_df['LastUni_2'].apply(check_top_15_unis)
ru_df['Top20LastPlace'] = ru_df['LastPlace'].apply(check_top_20_places)
ru_df['Top20LastPlace_2'] = ru_df['LastPlace_2'].apply(check_top_20_places)
ru_df['N_places'] = ru_df['N_places'].astype(float)
ru_df['ExpPeriod_Months'] = ru_df['ExpPeriod_Months'].astype(float)

In [ ]:
ru_df['WorkExpCoef'] = ru_df['N_places'] / ru_df['ExpPeriod_Months']

In [ ]:
ru_df['LastExp_Months'] = ru_df['LastExp_Months'].fillna(0)
ru_df['LastExp_2_Months'] = ru_df['LastExp_2_Months'].fillna(0)
ru_df['ExpPeriod_Months'] = ru_df['ExpPeriod_Months'].fillna(0)

In [ ]:
ru_df['HasSalary'] = np.where(np.isnan(ru_df['Salary']), 0, 1)
ru_df['Top12Uni'] = np.where(((ru_df['Top12LastUni'] == 1) | (ru_df['Top12LastUni_2'] == 1)), 1, 0)
ru_df['Top20Place'] = np.where(((ru_df['Top20LastPlace'] == 1) | (ru_df['Top20LastPlace_2'] == 1)), 1, 0)
ru_df['ExpPeriod_Years'] = ru_df['ExpPeriod_Months'] / 12
ru_df['LastExp_Years'] = ru_df['LastExp_Months'] / 12
ru_df['LastExp_2_Years'] = ru_df['LastExp_2_Months'] / 12
ru_df['N_extra_langs'] = ru_df['N_langs'] - 1

In [ ]:
langs_df = pd.json_normalize(ru_df['Langs'])
langs_df = langs_df.fillna(0)
work_rights_df = pd.json_normalize(ru_df['WorkRights'])
work_rights_df = work_rights_df.fillna(0)
work_schedule_df = pd.json_normalize(ru_df['WorkSchedule'])
work_schedule_df = work_schedule_df.fillna(0)
citizenship_df = pd.json_normalize(ru_df['Citizenship'])
citizenship_df = citizenship_df.fillna(0)
education_df = pd.json_normalize(ru_df['EduLevel'])

In [ ]:
temp_df = pd.concat([ru_df, langs_df, work_rights_df, citizenship_df, work_schedule_df, education_df], axis=1)
ru_df = temp_df
ru_df = ru_df.drop(
  ['Langs', 'ResumeLang', 'WorkRights', 'Citizenship', 'Top12LastUni', 'Top12LastUni_2', 
   'WorkExp', 'Time2Work', 'WorkSchedule', 'Top20LastPlace', 'Top20LastPlace_2', 'N_langs', 
   'EduLevel', 'WorkType'], 
  axis=1)

In [ ]:
ru_df

,Title,SpecCat,ExpPeriod_Months,Salary,Age,Gender,City,N_places,N_places_last_year,LastPlace,LastPlace_2,LastPos,LastPos_2,LastExp_Months,LastExp_2_Months,LastExpDesc,LastExpDesc_2,Description,Tags,Graduation,LastUni,LastUni_2,N_Unis,URL,FullTimeWork,Time2Work_Any,WorkExpCoef,HasSalary,Top12Uni,Top20Place,ExpPeriod_Years,LastExp_Years,LastExp_2_Years,N_extra_langs,Language_EN_Advanced,Language_RU_Native,WorkRights_Russia,WorkRights_Kazakhstan,WorkRights_Belarus,Citizenship_Russia,Citizenship_Kazakhstan,Citizenship_Belarus,WorkSchedule_FullDay,WorkSchedule_Remote,hasHigherSchoolFinished,hasBachelorDeg,hasMastersDeg,hasDoctorDeg
0,Финансовый аналитик,"Финансовый менеджер, Финансовый аналитик, инве...",0.0,NaN,35.0,1.0,Москва,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Уверенный пользователь ПК\r\nКоммуникабельност...,,NaN,ФА -- Финансовая академия при Правительстве РФ,NaN,1.0,https://hh.ru/resume/c4912d470004c8cba90039ed1...,1,1,NaN,0,0,0,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN
1,Финансовый аналитик,"Финансовый аналитик, инвестиционный аналитик, ...",0.0,NaN,21.0,0.0,Москва,1.0,1.0,Air Liquide,NaN,Бухгалтер по затратам,NaN,29.0,0.0,- корректное и своевременное отражение бухгалт...,NaN,Интересует работа с отчетами и цифрами. В бухг...,"Аналитическое мышление, Точность и внимательно...",NaN,Российский экономический университет им. Г.В. ...,ВФ РЭУ им.Плеханова,2.0,https://hh.ru/resume/3a2836470007ebfcf00039ed1...,1,1,NaN,0,0,0,0.000000,2.416667,0.000000,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN
2,"Менеджер проектов, финансовый аналитик",Руководитель проектов,186.0,150000.0,44.0,0.0,Москва,4.0,1.0,Good Foods,"Emerald resorts, Bulgaria",Финансовый аналитик,Финансовый директор,40.0,49.0,"Создание и реализация проекта с нуля, полный ...",Полный финансовый контроль пятизвездочного оте...,"Исполнительная, ответственная, целеустремленна...",Исполнительность,NaN,Московская государственная финансовая академия,Красноярский государственный торгово-экономиче...,2.0,https://hh.ru/resume/bb88768d000756bc0e0039ed1...,1,1,0.021505,1,0,0,15.500000,3.333333,4.083333,2.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN
3,Ведущий экономист финансового отдела,Экономист,269.0,130000.0,44.0,1.0,Москва,5.0,1.0,"ООО ""Итепло""","Вэб Медиа Сервисез, ЗАО",Финансовый аналитик,Руководитель отдела логистики и закупок,109.0,7.0,"Формирование бюджета, включая 8 обособленных п...","Организация, контроль и планирование работы по...",Знание компьютера на уровне эксперт;\r\nMicros...,"Ответственность, Высокий уровень грамотности, ...",Практический курс 1С Бухгалтерия 8.0,Финансовый Университет при Правительтве РФ,Томский Университет Систем Управления и Радиоэ...,2.0,https://hh.ru/resume/329c60e500011c9d6b0039ed1...,1,1,0.018587,1,0,0,22.416667,9.083333,0.583333,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN
4,Методолог по бюджетированию и управленческому ...,Финансовый менеджер,327.0,NaN,51.0,0.0,Москва,7.0,1.0,Индивидуальное предпринимательство / частная п...,ООО Агрохолдинг,Финансовый аналитик,Главный специалист по бюджетированию и управле...,43.0,4.0,В последнее время в связи с пандемией занялась...,"Постановка управленческого учета , внедрение п...","Имею опыт работы на производстве, оптовой и ро...","Управление затратами, Бюджетирование, Финансов...",Институт русского предпринимательства,иркутский институт народного хозяйства,NaN,1.0,https://hh.ru/resume/75b340880005a583570039ed1...,1,0,0.021407,0,0,0,27.250000,3.583333,0.333333,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33101,Бухгалтер,Бухгалтер,86.0,NaN,37.0,0.0,Москва,4.0,0.0,"ФГБОУ ВО ""РГАУ-МСХА имени К.А. Тимирязева""",ООО ЛИЕН,Начальник отдела подготовки научных и научно-п...,Помощник руководителя с функциями финансо

In [ ]:
ru_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33106 entries, 0 to 33105
Data columns (total 48 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Title                    33106 non-null  object 
 1   SpecCat                  33106 non-null  object 
 2   ExpPeriod_Months         33106 non-null  float64
 3   Salary                   16664 non-null  float64
 4   Age                      31444 non-null  float64
 5   Gender                   33033 non-null  float64
 6   City                     33106 non-null  object 
 7   N_places                 32591 non-null  float64
 8   N_places_last_year       33106 non-null  float64
 9   LastPlace                32579 non-null  object 
 10  LastPlace_2              29767 non-null  object 
 11  LastPos                  32579 non-null  object 
 12  LastPos_2                29767 non-null  object 
 13  LastExp_Months           33106 non-null  float64
 14  LastExp_2_Months      

In [ ]:
ru_df.to_csv('ru_resumes.csv', index=False)

In [ ]:
ru_train_df = ru_df.head(10000)

In [ ]:
ru_train_df.to_excel('ru_resumes_train.xlsx')

/opt/anaconda3/lib/python3.9/site-packages/xlsxwriter/worksheet.py:1170: UserWarning: Ignoring URL 'http://m.my.mail.ru/mail/mpr0xy/photo

Портфолио
----------------------------------------

Проведение%20презентаций,%20организация%20мероприятий,%20работа%20с%20пресс%20релизы,%20аккредитация%20СМИ,%20отчетность,%20постановка%20задач%20и%20мониторинг%20качества%20ее%20выполнения,%20высокие%20коммуникативные%20навыки,%20разработка%20креативных%20концепций,%20слоганов,%20работа%20с%20клиентами.
планирование%20маркетинговых%20кампаний,%20маркетинговые%20коммуникации,%20управление%20проектами,%20медиапланирование.
Администрирование%20и%20развитие%20корпоративных%20сайтов%20и%20интернет%20магазинов.
Web%20аналитика%20-%20настройка%20целей%20и%20анализ%20статистики%20сайта%20(в%20Яндекс%20Метрика,%20Google%20Analytics).
Сотрудничество%20с%20рекламными%20и%20медиа%20агентствами,%20типографиями,%20организациями%20по%20изготовлению%20различной%20рекламной%20продукции.
Глубокие%20знания%20в%20боль

In [ ]:
import nltk
from nltk.corpus import stopwords
import re
import string
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('snowball_data')
nltk.download('perluniprops')
nltk.download('universal_tagset')
nltk.download('nonbreaking_prefixes')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danilakokin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/danilakokin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package snowball_data to
[nltk_data]     /Users/danilakokin/nltk_data...
[nltk_data]   Package snowball_data is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /Users/danilakokin/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/danilakokin/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /Users/danilakokin/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dani

True

In [ ]:
ru_df

,Title,SpecCat,ExpPeriod_Months,Salary,Age,Gender,City,N_places,N_places_last_year,LastPlace,LastPlace_2,LastPos,LastPos_2,LastExp_Months,LastExp_2_Months,LastExpDesc,LastExpDesc_2,Description,Tags,Graduation,LastUni,LastUni_2,N_Unis,URL,FullTimeWork,Time2Work_Any,WorkExpCoef,HasSalary,Top12Uni,Top20Place,ExpPeriod_Years,LastExp_Years,LastExp_2_Years,N_extra_langs,Language_EN_Advanced,Language_RU_Native,WorkRights_Russia,WorkRights_Kazakhstan,WorkRights_Belarus,Citizenship_Russia,Citizenship_Kazakhstan,Citizenship_Belarus,WorkSchedule_FullDay,WorkSchedule_Remote,hasHigherSchoolFinished,hasBachelorDeg,hasMastersDeg,hasDoctorDeg
0,Финансовый аналитик,"Финансовый менеджер, Финансовый аналитик, инве...",0.0,NaN,35.0,1.0,Москва,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Уверенный пользователь ПК\r\nКоммуникабельност...,,NaN,ФА -- Финансовая академия при Правительстве РФ,NaN,1.0,https://hh.ru/resume/c4912d470004c8cba90039ed1...,1,1,NaN,0,0,0,0.000000,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN
1,Финансовый аналитик,"Финансовый аналитик, инвестиционный аналитик, ...",0.0,NaN,21.0,0.0,Москва,1.0,1.0,Air Liquide,NaN,Бухгалтер по затратам,NaN,29.0,0.0,- корректное и своевременное отражение бухгалт...,NaN,Интересует работа с отчетами и цифрами. В бухг...,"Аналитическое мышление, Точность и внимательно...",NaN,Российский экономический университет им. Г.В. ...,ВФ РЭУ им.Плеханова,2.0,https://hh.ru/resume/3a2836470007ebfcf00039ed1...,1,1,NaN,0,0,0,0.000000,2.416667,0.000000,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN
2,"Менеджер проектов, финансовый аналитик",Руководитель проектов,186.0,150000.0,44.0,0.0,Москва,4.0,1.0,Good Foods,"Emerald resorts, Bulgaria",Финансовый аналитик,Финансовый директор,40.0,49.0,"Создание и реализация проекта с нуля, полный ...",Полный финансовый контроль пятизвездочного оте...,"Исполнительная, ответственная, целеустремленна...",Исполнительность,NaN,Московская государственная финансовая академия,Красноярский государственный торгово-экономиче...,2.0,https://hh.ru/resume/bb88768d000756bc0e0039ed1...,1,1,0.021505,1,0,0,15.500000,3.333333,4.083333,2.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN
3,Ведущий экономист финансового отдела,Экономист,269.0,130000.0,44.0,1.0,Москва,5.0,1.0,"ООО ""Итепло""","Вэб Медиа Сервисез, ЗАО",Финансовый аналитик,Руководитель отдела логистики и закупок,109.0,7.0,"Формирование бюджета, включая 8 обособленных п...","Организация, контроль и планирование работы по...",Знание компьютера на уровне эксперт;\r\nMicros...,"Ответственность, Высокий уровень грамотности, ...",Практический курс 1С Бухгалтерия 8.0,Финансовый Университет при Правительтве РФ,Томский Университет Систем Управления и Радиоэ...,2.0,https://hh.ru/resume/329c60e500011c9d6b0039ed1...,1,1,0.018587,1,0,0,22.416667,9.083333,0.583333,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN
4,Методолог по бюджетированию и управленческому ...,Финансовый менеджер,327.0,NaN,51.0,0.0,Москва,7.0,1.0,Индивидуальное предпринимательство / частная п...,ООО Агрохолдинг,Финансовый аналитик,Главный специалист по бюджетированию и управле...,43.0,4.0,В последнее время в связи с пандемией занялась...,"Постановка управленческого учета , внедрение п...","Имею опыт работы на производстве, оптовой и ро...","Управление затратами, Бюджетирование, Финансов...",Институт русского предпринимательства,иркутский институт народного хозяйства,NaN,1.0,https://hh.ru/resume/75b340880005a583570039ed1...,1,0,0.021407,0,0,0,27.250000,3.583333,0.333333,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33101,Бухгалтер,Бухгалтер,86.0,NaN,37.0,0.0,Москва,4.0,0.0,"ФГБОУ ВО ""РГАУ-МСХА имени К.А. Тимирязева""",ООО ЛИЕН,Начальник отдела подготовки научных и научно-п...,Помощник руководителя с функциями финансо

In [ ]:
def stemmer(text):
  stemmer = SnowballStemmer(language='russian')
  tokenizer = nltk.tokenize.WhitespaceTokenizer()
  tokens = [stemmer.stem(w) for w in tokenizer.tokenize(text)]

  return ' '.join(tokens)

In [ ]:
def clean_text(text):

  text = text.lower() 
  text = re.sub(r'\n', '', text)
  translator = str.maketrans('', '', string.punctuation)
  text = text.translate(translator)
  text = text.split()
  useless_words = stopwords.words("english") + stopwords.words("russian") + ['добрый', 'день', 'hi', 'также']
  text_filtered = [word for word in text if not word in useless_words]
  final_string = ' '.join(text_filtered)

  return final_string

In [ ]:
compare_df = pd.DataFrame()
compare_df['Description'] = df['Description']
compare = compare_df.dropna()

In [ ]:
desc_df = df['Description'].dropna()

In [ ]:
compare_df['Cleaned'] = desc_df.apply(clean_text)
compare_df = compare_df.dropna()
compare_df['CleanedAndStemmed'] = compare_df['Cleaned'].apply(stemmer)

In [ ]:
compare_df

,Description,Cleaned,CleanedAndStemmed
0,- с 2019 по 2022 был небольшой собственный биз...,2019 2022 небольшой собственный бизнес связанн...,2019 2022 небольш собствен бизнес связа финанс...
2,Высокий уровень ответственности и самоконтроля...,высокий уровень ответственности самоконтроля н...,высок уровен ответствен самоконтрол нацелен ре...
3,Параллельно с обучением в Высшей Школе Экономи...,параллельно обучением высшей школе экономики у...,параллельн обучен высш школ экономик успешн то...
4,"Я молодой, амбициозный человек, имеющий хороши...",молодой амбициозный человек имеющий хорошие те...,молод амбициозн человек имеющ хорош техническ ...
5,•\tИмею опыт работы в финансовой сфере 18 лет\...,• имею опыт работы финансовой сфере 18 лет• об...,• им оп работ финансов сфер 18 лет• образован ...
...,...,...,...
54863,Публикации:\r\nИмеются научные статьи в журнал...,публикации имеются научные статьи журналах меж...,публикац имеют научн стат журнал международн б...
54864,"Аналитическое мышление, организаторские способ...",аналитическое мышление организаторские способн...,аналитическ мышлен организаторск способн умен ...
54865,Активная жизненная позиция(во время учебы в ко...,активная жизненная позицияво время учебы колле...,активн жизнен позицияв врем учеб колледж работ...
54866,•\tРазвитые управленческие и организаторские н...,• развитые управленческие организаторские навы...,• развит управленческ организаторск навык • ра...


In [ ]:
compare_df.to_csv('descriptions.csv', index=False)